In [ ]:
import os, pathlib, zipfile, re
import urllib.request, requests
from IPython.display import HTML, clear_output
from urllib.parse import urlparse

import IPython
from google.colab import output

#####################################
USE_FREE_TOKEN = True 
REGION = "US"
# Aria2 Service
Aria2_rpc = True
Ariang_WEBUI = True
#@markdown <img src='https://i.imgur.com/ZeWgURH.png' height="70"/>
UPLOAD = "OneDrive" #@param ["OneDrive", "Google Drive"]
TOKEN = ""
#@markdown &emsp;<b>KEEPALIVE SCRIPT:<b><br><small>&emsp;&emsp;function ClickConnect(){<br>&emsp;&emsp;&emsp;console.log("Working");<br>&emsp;&emsp;&emsp;document.querySelector("colab-toolbar-button").click()<br>&emsp;&emsp;&emsp;}setInterval(ClickConnect,60000)<br><br></small>
#@markdown &emsp;<b>AUTO-RECONNECT SCRIPT:<b><br><small>&emsp;&emsp;function ClickConnect(){<br>&emsp;&emsp;&emsp;console.log('Reconnection in progress')<br>&emsp;&emsp;&emsp;if (document.querySelector('#top-toolbar > colab-connect-button')<br>&emsp;&emsp;&emsp;&emsp;.shadowRoot.querySelector('#connect')<br>&emsp;&emsp;&emsp;&emsp;.textContent<br>&emsp;&emsp;&emsp;&emsp;.replace(/[\n\r]+|[\s]{2,}/g, '').trim() == 'Reconnect') {<br>&emsp;&emsp;&emsp;&emsp;document.querySelector('#top-toolbar > colab-connect-button').shadowRoot.querySelector('#connect').click();<br>&emsp;&emsp;&emsp;} else {<br>&emsp;&emsp;&emsp;&emsp;console.log('Nothing to click');<br>&emsp;&emsp;&emsp;}<br>&emsp;&emsp;}<br>&emsp;&emsp;setInterval(ClickConnect, 60000)</small>
PORT_FORWARD = "argotunnel"
# OUTPUT_DIR = ""
PORT = 8221
HOME = os.path.expanduser("~")

!curl https://rclone.org/install.sh | sudo bash
!rclone config file

if UPLOAD == "Google Drive":
    RCLONE_URL = "https://gist.githubusercontent.com/pctechkid/9e4901d8dfca0895623e1eb6369a2fe5/raw/245f97e22456f550c1053761e6ecaac5bba9e36f/rclone.conf"
elif UPLOAD == "OneDrive":
    RCLONE_URL = "https://gist.githubusercontent.com/pctechkid/32b906817ee12c1fbfd3a5fa6f1ff585/raw/5cb65dcf0f90db8cf35fd6271a9fce3aec5882e6/rclone.conf"

# Download aria2 configuration file and overwrite it if it exists
!wget -O /root/.config/rclone/aria2.conf https://gist.githubusercontent.com/pctechkid/3f919f42bf69d8b1e91bca75d0ebca4e/raw/468420f864ddd551636efb8f91ef769049e2b042/aria2.conf

# Set permissions for aria2.conf to 777 (read, write, and execute for everyone)
!chmod 777 /root/.config/rclone/aria2.conf

# Download a file defined by the "RCLONE_URL" variable and overwrite it if it exists
!wget -O /root/.config/rclone/rclone.conf $RCLONE_URL

# Set permissions for rclone.conf to 777 (read, write, and execute for everyone)
!chmod 777 /root/.config/rclone/rclone.conf

# Download transfer.sh script and overwrite it if it exists
!wget -O /root/.config/rclone/transfer.sh https://gist.githubusercontent.com/pctechkid/de60f085d6202ba087c0fa77598ce974/raw/ce665f15f5a61a84f850c6232da0e2bf1f651c36/transfer.sh

# Set permissions for transfer.sh to 777 (read, write, and execute for everyone)
!chmod 777 /root/.config/rclone/transfer.sh


if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/pctechkid/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    CWD,
    displayUrl,
    findProcess,
    findPackageR
)

loadingAn()

# Ariang SETUP
runSh('apt install -y aria2')
pathlib.Path('ariang').mkdir(mode=0o777, exist_ok=True)
pathlib.Path('downloads').mkdir(mode=0o777, exist_ok=True)

# github latest releases tag define
def latestTag(link):
  import re
  from urllib.request import urlopen
  htmlF = urlopen(link+"/releases/latest").read().decode('UTF-8')
  return re.findall(r'.+\/tag\/([.0-9A-Za-z]+)".+/', htmlF)[0]

# Downloading latest version of ariang
if not os.path.exists("ariang/index.html"):
  # BASE_URL = r"https://github.com/mayswind/AriaNg"
  # LATEST_TAG = latestTag(BASE_URL)
  # urlF = f'{BASE_URL}/releases/download/{LATEST_TAG}/' \
  #             f'AriaNg-{LATEST_TAG}-AllInOne.zip'
  urllib.request.urlretrieve(
      findPackageR('mayswind/AriaNg', 'AllInOne.zip'),
      'ariang/new.zip')
  with zipfile.ZipFile('ariang/new.zip', 'r') as zip_ref:
      zip_ref.extractall('ariang')
  try:
    pathlib.Path('ariang/new.zip').unlink()
  except FileNotFoundError:
    pass

# START_ARIANG_WEBUI_AND_ARIA2_RPC
try:
  if not OUTPUT_DIR:
    OUTPUT_DIR = f"downloads/"
  elif not os.path.exists(OUTPUT_DIR):
    clear_output()
    print("Error: Your set path not found! Create path!")
    exx()
except:
  OUTPUT_DIR = f"{CWD}/downloads/"
if Aria2_rpc:
  if not findProcess("aria2c", "--enable-rpc"):
    try:
      trackers = requests.get(
          "https://raw.githubusercontent.com/ngosang/trackerslist/master/trackers_best.txt"
      ).text.replace("\n\n", ",")
      cmdC = r"aria2c --enable-rpc --rpc-listen-port=6800 -D " \
            fr"-d {OUTPUT_DIR} " \
            r"--conf-path=/root/.config/rclone/aria2.conf " \
            fr"--bt-tracker={trackers} " \
            r" &"
      runSh(
          cmdC,
          shell=True
      )
    except:
      print("aria2 rpc not enabling. Try again later!")

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
if Aria2_rpc:
  Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['Aria2_rpc', 6800, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/aria2.yml", 5042]
)
  data = Server.start('Aria2_rpc', displayB=False)
  Host = urlparse(data['url']).hostname
  port = "80"
  clear_output()
if Ariang_WEBUI:
  if Aria2_rpc:
    filePath = 'ariang/index.html'
    with open(filePath, 'r+') as f:
      read_data = f.read()
      f.seek(0)
      f.truncate(0)
      read_data = re.sub(r'(rpcHost:"\w+.")|rpcHost:""', 
                        f'rpcHost:"{Host}"', read_data)
      read_data = re.sub(r'protocol:"\w+."', r'protocol:"ws"', read_data)
      read_data = re.sub(r'rpcPort:"\d+."', f'rpcPort:"{port}"', read_data)
      f.write(read_data)
  try:
    urllib.request.urlopen(f"http://localhost:{PORT}")
  except:
    runSh(f"python3 -m http.server {PORT} &", shell=True, cd="ariang/")
  
  Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['Ariang', PORT, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/ariang.yml", 5043]
)
  data2 = Server.start('Ariang', displayB=False)
  data2['url'] = urlparse(data2['url'])._replace(scheme='http').geturl()
  ariaNgLink = data2['url']

if Aria2_rpc:
  display(HTML("""<!doctypehtml><meta content="width=device-width,initial-scale=1"name=viewport><style>.tooltip{position:relative;display:inline-block}.tooltip .tooltiptext{visibility:hidden;width:140px;background-color:#555;color:#fff;text-align:center;border-radius:6px;padding:5px;position:absolute;z-index:1;bottom:150%;left:50%;margin-left:-75px;opacity:0;transition:opacity .3s}.tooltip .tooltiptext::after{content:"";position:absolute;top:100%;left:50%;margin-left:-5px;border-width:5px;border-style:solid;border-color:#555 transparent transparent transparent}.tooltip:hover .tooltiptext{visibility:visible;opacity:1}</style><input hidden id=myInput value="""+Host+"""> <a href="http://www.ronnel-downloads.serv00.net/#!/settings/rpc/set?protocol=ws&host="""+Host+"""&port=80&interface=jsonrpc" target=_blank><button>Ronnel Downloads Portal</button></a><br><br><div class=tooltip><button onclick=myFunction() onmouseout=outFunc()><span class=tooltiptext id=myTooltip>Copy to clipboard</span> WebSocket</button></div><script>function myFunction(){var e=document.getElementById("myInput");e.select(),e.setSelectionRange(0,99999),navigator.clipboard.writeText(e.value),document.getElementById("myTooltip").innerHTML="Success!"}function outFunc(){document.getElementById("myTooltip").innerHTML="Click to copy"}</script>"""))